<a href="https://colab.research.google.com/github/sierrarowley/research_fall2020/blob/master/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import string
import numpy as np

xbound = 10
ybound = 10
point_bound = 1000

def generatedata():
  db = np.empty((xbound, ybound), dtype=object)
  # create random number of points
  for n in range(0, random.randint(1, point_bound)):
    x = random.randint(0, xbound-1)
    y = random.randint(0, ybound-1)

    # encrypt value of point
    randstring = ''.join(random.choices(string.ascii_letters + string.digits, k=100))
    enc = hash(randstring + str(x) + str(y))

    # store encrypted point in the database
    if db[x, y] == None:
      db[x, y] = [enc]
    # at most 10 values per point
    elif len(db[x, y]) < 10:
      db[x, y].append(enc)
  
  return db

The database is classified as either sparse (0) or dense (1). produce_leakage() performs every possible query of the database and saves the results from those queries in a list. The list is returned at the end.

In [2]:
def produce_leakage(db):
  leakage = []
  dense_flag = 1
  querylen = 0
  # all possible size rectangle queries
  for l in range(1, xbound+1):
    for w in range(1, ybound+1):
      # all possible starting points of the rectangle query
      for x in range(0, xbound):
        # dont perform query if it is out of bounds
        if x+l > xbound:
          break
        for y in range(0, ybound):
          if y+w > ybound:
            break

          # if any point in database is None, then it is not dense
          if db[x, y] == None:
            dense_flag = 0
          
          curr_query = []
          # loop through current query in the database
          for i in range(x, x+l):
            for j in range(y, y+w):
              if db[i, j] != None:
                curr_query.extend(db[i, j])

          # add query leakage to entire list
          if len(curr_query) != 0:
            leakage.append(curr_query)
          # keep track of longest query length for creating input array
          if len(curr_query) > querylen:
            querylen = len(curr_query)
  
  # shuffle to make location of queries less obvious
  random.shuffle(leakage)
  return (leakage, dense_flag, querylen)

create_database() calls generatedata() and produce_leakage(), which creates the database and the search query leakages from that database. It then takes the returned list of leakages and transforms it into a 2D numpy array with dimensions (number of queries, length of longest query). All queries with a length shorter than the longest are padded with 0's at the end. Each index of the array represents hashed value of a point in a query. create_database() returns the numpy array, which represents one input to the neural net, and the classification of the database.

In [3]:
def create_database():
  leakage, c, querylen = produce_leakage(generatedata())
  numqueries = len(leakage)
  
  input = np.empty((numqueries, querylen))
  for x in range(0, numqueries):
    q = np.array(leakage[x]).reshape(-1,)
    input[x] = np.pad(q, (0, querylen-len(q)), 'constant', constant_values=(0, 0))
  
  return (input, c)

create_nn_data() creates a specified number of databases that will be used in the training and testing sets for the neural net. It pads each input array (from create_database) so they are all of the same size. It returns an array of inputs of size (number of inputs, number of queries, number of items in query) and an array of the labels.

In [4]:
def create_nn_data(num_inputs):
  data = []
  labels = np.zeros((num_inputs))
  numqueries = 0
  querylen = 0
  # create num_inputs databases for input
  for n in range(0, num_inputs):
    database, label = create_database()
    data.append(database)
    labels[n] = label

    # find largest shape to later pad all databases to be same size
    if database.shape[0] > numqueries:
      numqueries = database.shape[0]
    if database.shape[1] > querylen:
      querylen = database.shape[1]
  
  inputs = np.empty((num_inputs, numqueries, querylen))

  # pad each database
  for i in range(0, len(data)):
    d = data[i]
    d = np.pad(d, ((0, max(numqueries - d.shape[0], 0)), (0, max(querylen - d.shape[1], 0))), 'constant', constant_values=(0, 0))
    inputs[i] = d

  return inputs, labels

Time to make the neural net! I will be using tensorflow with keras

In [5]:
import tensorflow as tf
from tensorflow import keras

In [16]:
n = 100
inputs, labels = create_nn_data(n)

train_inputs = inputs[0:int(n*.8), :, :, None]
train_labels = labels[0:int(n*.8)]
test_inputs = inputs[int(n*.8):n, :, :, None]
test_labels = labels[int(n*.8):n]

print("train inputs: ", train_inputs.shape, "train labels: ", train_labels.shape)
print("test inputs: ", test_inputs.shape, "test labels: ", test_labels.shape)

model = keras.Sequential([
    # keras.layers.Conv2D(32, 5, strides=(2, 2), padding='same', activation='relu', kernel_regularizer='L2'),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="same"),
    # keras.layers.Conv2D(64, 5, strides=(2, 2), padding='same', activation='relu', kernel_regularizer='L2'),
    keras.layers.Flatten(input_shape=(train_inputs.shape[1], train_inputs.shape[2])),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_inputs, train_labels, epochs=3)

test_loss, test_acc = model.evaluate(test_inputs,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

train inputs:  (80, 3025, 877, 1) train labels:  (80,)
test inputs:  (20, 3025, 877, 1) test labels:  (20,)
Epoch 1/3
3/3 [==============================] - 1s 453ms/step - loss: 1359039802640433152.0000 - accuracy: 0.5250
Epoch 2/3
3/3 [==============================] - 1s 453ms/step - loss: nan - accuracy: 0.6375
Epoch 3/3
3/3 [==============================] - 1s 406ms/step - loss: nan - accuracy: 0.5000
1/1 - 0s - loss: nan - accuracy: 0.6000

Test accuracy: 0.6000000238418579


In [14]:
model = keras.Sequential([
    # keras.layers.Conv2D(32, 5, strides=(2, 2), padding='same', activation='relu', kernel_regularizer='L2'),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="same"),
    # keras.layers.Conv2D(64, 5, strides=(2, 2), padding='same', activation='relu', kernel_regularizer='L2'),
    keras.layers.Flatten(input_shape=(train_inputs.shape[1], train_inputs.shape[2])),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_inputs, train_labels, epochs=3)

test_loss, test_acc = model.evaluate(test_inputs,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

Epoch 1/3
3/3 [==============================] - 1s 427ms/step - loss: 1944169242056196096.0000 - accuracy: 0.5625
Epoch 2/3
3/3 [==============================] - 1s 425ms/step - loss: 1472428176205414400.0000 - accuracy: 0.8375
Epoch 3/3
3/3 [==============================] - 1s 373ms/step - loss: 645667999885295616.0000 - accuracy: 0.9000
1/1 - 0s - loss: 7945690047355813888.0000 - accuracy: 0.6500

Test accuracy: 0.6499999761581421
